In [53]:
# -------
# IMPORT LIBRAIRIES
# -------
import numpy as np
import pandas as pd
from python_module.pricing_model import SABRModel
from python_module.tools import maximize_with_bounds

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.float_format = '{:,.2f}'.format

In [54]:
# -------
# GENERATE MARKET DATA AND GREEKS
# -------

# Pricing parameters
F     = 5000
alpha = 0.25
beta  = 1.0
rho   = -0.5
nu    = 1.5
r     = 0
bd    = 20
T     = bd / 250

# Scenario parameters
slide_to_compute = [-0.3, -0.1, -0.05, -0.04, -0.03, -0.02, -0.01, 0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.3]
slide_to_compute = [-0.05, -0.04, -0.03, -0.02, -0.01, 0.01, 0.02, 0.03, 0.04, 0.05]


strike_list = []
for delta in [-0.01, -0.1, -0.2, -0.3, -0.4, 0.5, 0.4, 0.3, 0.2, 0.1, 0.01]:
    option_type = 'call' if delta > 0 else 'put'
    K = SABRModel.solve_delta_strike(F=F, T=T, alpha=alpha, beta=beta, rho=rho, nu=nu, r=r, option_type=option_type, target_delta=delta)
    K = int(K)
    strike_list.append(K)

#if min(strike_list) > F*0.8:
#    strike_list.append(int(F*0.8))

#if F*1.2 > max(strike_list):
#    strike_list.append(int(F*1.2))

strike_list = sorted(strike_list)

market_data_list = list()
for K in strike_list:

    option_type = 'call' if K >= F else 'put'
    
    market_pricing_results = SABRModel.compute_option(F, K, T, alpha, beta, rho, nu, r, option_type, slide_list=slide_to_compute)

    market_data_list.append({
        'symbol': f"{bd}T_{K}K_{option_type}",
        'option_type': option_type, 
        'time_to_maturity': bd, 
        'F':F, 'r': r, 'F': F, 'K': K, 'T': T, **market_pricing_results})

market_data_df = pd.DataFrame(market_data_list)
market_data_df = market_data_df.set_index('symbol')

In [55]:
market_data_df

,option_type,time_to_maturity,F,r,K,T,IV,price,delta,gamma,vega,theta,vanna,volga,-0.05,-0.04,-0.03,-0.02,-0.01,0.01,0.02,0.03,0.04,0.05
symbol,,,,,,,,,,,,,,,,,,,,,,,,
20T_4222K_put,put,20,5000,0,4222,0.08,0.33,5.81,-0.03,0.00,0.99,-0.81,-0.38,996.03,9.53,5.95,3.35,1.58,0.50,-0.03,0.33,0.99,1.89,2.96
20T_4513K_put,put,20,5000,0,4513,0.08,0.30,21.34,-0.10,0.00,2.54,-1.88,-0.71,"1,276.99",23.16,14.86,8.58,4.13,1.33,-0.06,0.96,2.89,5.58,8.88
20T_4703K_put,put,20,5000,0,4703,0.08,0.28,47.24,-0.21,0.00,4.03,-2.79,-0.76,884.53,34.98,23.00,13.61,6.72,2.24,-0.17,1.53,4.88,9.66,15.66
20T_4820K_put,put,20,5000,0,4820,0.08,0.27,74.64,-0.30,0.00,4.92,-3.27,-0.59,434.81,40.99,27.44,16.54,8.35,2.85,-0.31,1.77,6.05,12.32,20.33
20T_4922K_put,put,20,5000,0,4922,0.08,0.26,108.49,-0.40,0.00,5.47,-3.52,-0.27,96.04,43.69,29.75,18.27,9.42,3.31,-0.49,1.77,6.68,14.07,23.73
20T_5000K_call,call,20,5000,0,5000,0.08,0.25,141.81,0.51,0.00,5.64,-3.54,0.06,-2.84,43.45,29.98,18.70,9.81,3.53,-0.67,1.57,6.71,14.64,25.23
20T_5102K_call,call,20,5000,0,5102,0.08,0.24,94.06,0.40,0.00,5.46,-3.33,0.52,188.36,39.86,28.00,17.83,9.60,3.57,-0.90,1.06,6.00,13.99,25.01
20T_5194K_call,call,20,5000,0,5194,0.08,0.24,61.48,0.30,0.00,4.90,-2.92,0.87,650.98,33.96,24.23,15.72,8.67,3.34,-1.06,0.42,4.70,11.98,22.39
20T_5291K_call,call,20,5000,0,5291,0.08,0.23,37.04,0.21,0.00,4.02,-2.35,1.08,"1,256.96",26.27,19.05,12.61,7.13,2.85,-1.13,-0.25,2.97,8.83,17.59


In [56]:
slide_df = market_data_df.loc[:, market_data_df.columns[14:]] 
theta = market_data_df['theta']
theta.name = 0
scenarios = pd.concat([slide_df, theta], axis=1)
scenarios = scenarios.transpose().sort_index().transpose()

In [57]:
scenarios

,-0.05,-0.04,-0.03,-0.02,-0.01,0.00,0.01,0.02,0.03,0.04,0.05
symbol,,,,,,,,,,,
20T_4222K_put,9.53,5.95,3.35,1.58,0.50,-0.81,-0.03,0.33,0.99,1.89,2.96
20T_4513K_put,23.16,14.86,8.58,4.13,1.33,-1.88,-0.06,0.96,2.89,5.58,8.88
20T_4703K_put,34.98,23.00,13.61,6.72,2.24,-2.79,-0.17,1.53,4.88,9.66,15.66
20T_4820K_put,40.99,27.44,16.54,8.35,2.85,-3.27,-0.31,1.77,6.05,12.32,20.33
20T_4922K_put,43.69,29.75,18.27,9.42,3.31,-3.52,-0.49,1.77,6.68,14.07,23.73
20T_5000K_call,43.45,29.98,18.70,9.81,3.53,-3.54,-0.67,1.57,6.71,14.64,25.23
20T_5102K_call,39.86,28.00,17.83,9.60,3.57,-3.33,-0.90,1.06,6.00,13.99,25.01
20T_5194K_call,33.96,24.23,15.72,8.67,3.34,-2.92,-1.06,0.42,4.70,11.98,22.39
20T_5291K_call,26.27,19.05,12.61,7.13,2.85,-2.35,-1.13,-0.25,2.97,8.83,17.59


In [63]:
target_scenario = 0.05
df = scenarios.drop(target_scenario, axis=1)
s = scenarios[target_scenario]

In [64]:
# Constrain weights between -0.5 and 0.8
w_opt, res = maximize_with_bounds(df=df, s=s, w_min=-10000, w_max=10000)

print("Optimal weights:\n", w_opt)
print("Maximized objective (w^T s):", float(w_opt.dot(s)))

Optimal weights:
 symbol
20T_4222K_put    -10,000.00
20T_4513K_put    -10,000.00
20T_4703K_put    -10,000.00
20T_4820K_put    -10,000.00
20T_4922K_put      8,293.72
20T_5000K_call    10,000.00
20T_5102K_call    10,000.00
20T_5194K_call    10,000.00
20T_5291K_call    -8,038.11
20T_5434K_call   -10,000.00
20T_5633K_call   -10,000.00
Name: weights, dtype: float64
Maximized objective (w^T s): 192203.21323885306


In [65]:
scenarios.multiply(w_opt, axis=0).sum(axis=0)

-0.05    27,564.88
-0.04    46,804.34
-0.03    44,642.02
-0.02    30,166.96
-0.01    12,461.63
0.00          0.00
0.01          0.00
0.02     17,787.89
0.03     56,242.31
0.04    115,388.19
0.05    192,203.21
dtype: float64